In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape
from tensorflow.keras.optimizers import Adam
# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
save_check_pt = './checkpoints/weather'
my_epoch = 1

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = cv2.imread(img_dir)
#         img = cv2.resize(img,(32,32))
#         X.append(img)
#     X = np.array(X)
#     return X

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L') # conver to grayscale images
        img = img.resize([32, 16])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X


def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(256, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(128, activation="relu"))
    model.compile(optimizer=Adam(0.01), loss='loss')
    model.summary()
    return model

def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(35, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(0.01), loss='mse')
    return model

def get_discriminator(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu',input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(35, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(optimizer=Adam(0.01), loss='mse')
    return model

base_images_dir, base_labels, base_table = load_ccpd(dataset_path['base'],mode='all')
base_images = readCcpdImg(base_images_dir)
base_labels = np.array(base_labels)
weather_images_dir, weather_labels, weather_table = load_ccpd(dataset_path['weather'],mode='all')
weather_images = readCcpdImg(weather_images_dir)
weather_labels = np.array(weather_labels)
weather_images_dir_t, weather_labels_t, weather_table_t = load_ccpd(dataset_path['weather'],mode='train')
weather_images_t = readCcpdImg(weather_images_dir_t)
weather_labels_t = np.array(weather_labels_t)
print(np.shape(base_images))
print(np.shape(weather_images))
print(np.shape(weather_images_t))
model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=1.0, optimizer=Adam(0.001),  metrics=["acc"],random_state=0)
model.fit(base_images, base_labels,weather_images, epochs=500,verbose=1)
model.summary()
model.score(weather_images, weather_labels)
model.save_weights(save_check_pt)

(8446, 512)
(3812, 512)
(762, 512)
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
Total params: 164,224
Trainable params: 164,224
Non-trainable params: 0
_________________________________________________________________
Epoch 1/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5768 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 2/500
358/358 [==============================] - 1s 2ms/step - loss: 154.9158 - acc: 0.0404 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 3/500
358/358 [==============================] - 1s 2ms/step - loss: 155.0813 - acc: 0.0414 - disc_loss: 15.9424 - disc_acc: 0.50

358/358 [==============================] - 1s 2ms/step - loss: 155.0199 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 55/500
358/358 [==============================] - 1s 2ms/step - loss: 154.2132 - acc: 0.0413 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 56/500
358/358 [==============================] - 1s 2ms/step - loss: 155.1716 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 57/500
358/358 [==============================] - 1s 2ms/step - loss: 155.2164 - acc: 0.0416 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 58/500
358/358 [==============================] - 1s 2ms/step - loss: 155.7622 - acc: 0.0414 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 59/500
358/358 [==============================] - 1s 2ms/step - loss: 155.1926 - acc: 0.0411 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 60/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5665 - acc: 0.0404 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 61/500
358/358 [==================

358/358 [==============================] - 1s 2ms/step - loss: 154.8595 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 113/500
358/358 [==============================] - 1s 2ms/step - loss: 154.1574 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 114/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3147 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 115/500
358/358 [==============================] - 1s 2ms/step - loss: 155.1942 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 116/500
358/358 [==============================] - 1s 2ms/step - loss: 155.0902 - acc: 0.0415 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 117/500
358/358 [==============================] - 1s 2ms/step - loss: 155.8562 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 118/500
358/358 [==============================] - 1s 2ms/step - loss: 155.8528 - acc: 0.0406 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 119/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 156.4923 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 171/500
358/358 [==============================] - 1s 2ms/step - loss: 155.1374 - acc: 0.0414 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 172/500
358/358 [==============================] - 1s 2ms/step - loss: 154.9655 - acc: 0.0410 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 173/500
358/358 [==============================] - 1s 2ms/step - loss: 154.9125 - acc: 0.0414 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 174/500
358/358 [==============================] - 1s 2ms/step - loss: 156.1547 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 175/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5826 - acc: 0.0405 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 176/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3527 - acc: 0.0418 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 177/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 155.4927 - acc: 0.0395 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 229/500
358/358 [==============================] - 1s 2ms/step - loss: 155.2373 - acc: 0.0406 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 230/500
358/358 [==============================] - 1s 2ms/step - loss: 155.6343 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 231/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3155 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 232/500
358/358 [==============================] - 1s 2ms/step - loss: 154.7216 - acc: 0.0415 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 233/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3542 - acc: 0.0411 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 234/500
358/358 [==============================] - 1s 2ms/step - loss: 155.9474 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 235/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 156.4089 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 287/500
358/358 [==============================] - 1s 2ms/step - loss: 155.7341 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 288/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4505 - acc: 0.0409 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 289/500
358/358 [==============================] - 1s 2ms/step - loss: 156.1590 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 290/500
358/358 [==============================] - 1s 2ms/step - loss: 155.1298 - acc: 0.0401 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 291/500
358/358 [==============================] - 1s 2ms/step - loss: 154.9278 - acc: 0.0406 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 292/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5547 - acc: 0.0412 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 293/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 155.9422 - acc: 0.0405 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 345/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4770 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 346/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5031 - acc: 0.0416 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 347/500
358/358 [==============================] - 1s 2ms/step - loss: 155.7744 - acc: 0.0412 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 348/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4643 - acc: 0.0405 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 349/500
358/358 [==============================] - 1s 2ms/step - loss: 154.8687 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 350/500
358/358 [==============================] - 1s 2ms/step - loss: 155.2672 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 351/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 155.3498 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 403/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4940 - acc: 0.0411 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 404/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3447 - acc: 0.0415 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 405/500
358/358 [==============================] - 1s 2ms/step - loss: 155.8398 - acc: 0.0405 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 406/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4622 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 407/500
358/358 [==============================] - 1s 2ms/step - loss: 156.0581 - acc: 0.0410 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 408/500
358/358 [==============================] - 1s 2ms/step - loss: 156.1477 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 409/500
358/358 [===========

358/358 [==============================] - 1s 2ms/step - loss: 156.1244 - acc: 0.0407 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 461/500
358/358 [==============================] - 1s 2ms/step - loss: 155.2148 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 462/500
358/358 [==============================] - 1s 2ms/step - loss: 155.4223 - acc: 0.0408 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 463/500
358/358 [==============================] - 1s 2ms/step - loss: 155.5927 - acc: 0.0415 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 464/500
358/358 [==============================] - 1s 2ms/step - loss: 155.3249 - acc: 0.0400 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 465/500
358/358 [==============================] - 1s 2ms/step - loss: 154.7457 - acc: 0.0411 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 466/500
358/358 [==============================] - 1s 2ms/step - loss: 155.2108 - acc: 0.0412 - disc_loss: 15.9424 - disc_acc: 0.5000
Epoch 467/500
358/358 [===========

In [2]:
# fig = plt.figure()
# img_h, img_w = 32,16
# fig.subplots_adjust(hspace=0.4, wspace=0.4)
# for i in range(6):
#     ax = fig.add_subplot(2, 3, i+1)
#     ax.imshow(weather_images[i].reshape([img_h, img_w]))
#model.save_weights('ckpt')
#dummy step to load model from checkpoint
#xt = np.zeros((3812, 512))
#nmd = DANN(lambda_=0.1, Xt=xt, metrics=["acc"], random_state=0)
#x = np.zeros((1,1))
#y = np.zeros(1)
#nmd.fit(x,y, epochs=0,verbose=1)
#nmd.summary()
#nmd.load_weights("ckpt")
#nmd.score(weather_images, weather_labels)
#print(nmd.predict(weather_images[0].reshape([-1,512])))